In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import pandas as pd
import numpy as np

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Tải mô hình và tokenizer đã lưu
model = AutoModelForSequenceClassification.from_pretrained('./deberta_model')
tokenizer = AutoTokenizer.from_pretrained('./deberta_model')

In [3]:
file_name = 'cleaned_arxiv_cs_papers_all.csv'
raw_df = pd.read_csv(file_name)

In [4]:
# Tiền xử lý: Ghép tiêu đề và tóm tắt lại
raw_df['text'] = raw_df['title'] + " " + raw_df['abstract']

# Chuyển nhãn thành các giá trị nhị phân (ví dụ: cho nhiều nhãn)
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(raw_df['categories'].apply(lambda x: x.split(', ')))

In [5]:
!pip install nltk

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Tải các tài nguyên cần thiết từ NLTK
nltk.download('punkt')  # Để sử dụng word_tokenize
nltk.download('stopwords')  # Để sử dụng stopwords
nltk.download('wordnet')  # Để sử dụng lemmatizer


# Khởi tạo lemmatizer và stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Tiền xử lý dữ liệu
raw_df['text'] = raw_df['text'].str.lower()  # Chuyển về chữ thường
raw_df['text'] = raw_df['text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))  # Loại bỏ ký tự đặc biệt
raw_df['text'] = raw_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))  # Loại bỏ stopwords
raw_df['text'] = raw_df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))  # Lemmatization

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_texts, val_texts, train_labels, val_labels = train_test_split(raw_df['text'], labels, test_size=0.2)

In [8]:
val_encodings = tokenizer(list(val_texts), padding=True, truncation=True, max_length=150)

In [9]:
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_labels))
val_dataloader = DataLoader(val_dataset, batch_size=8)

In [10]:
from sklearn.metrics import accuracy_score, f1_score, hamming_loss

# Huấn luyện mô hình
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

    # Đánh giá trên tập kiểm tra (validation)
model.eval()
correct_predictions = 0
total_predictions = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        # Forward pass (Chạy mô hình)
        outputs = model(input_ids)
        logits = outputs.logits

        # Tính dự đoán (điểm số > 0.5 cho mỗi lớp)
        preds = torch.sigmoid(logits) > 0.5  # Các giá trị đầu ra >= 0.5 được coi là nhãn đúng

        # Cộng dồn các kết quả
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.numel()  # Tổng số nhãn

        all_labels.append(labels.cpu().numpy())  # Chuyển labels sang numpy để tính accuracy
        all_preds.append(preds.cpu().numpy())

# Tính accuracy theo toàn bộ dataset
accuracy = accuracy_score(np.concatenate(all_labels), np.concatenate(all_preds))
print(f"Validation Accuracy: {accuracy:.4f}")


hamming = hamming_loss(np.concatenate(all_labels), np.concatenate(all_preds))
print(f"Hamming Loss: {hamming:.4f}")


f1 = f1_score(np.concatenate(all_labels), np.concatenate(all_preds), average='micro')
print(f"Validation F1-Score: {f1:.4f}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Validation Accuracy: 0.5091
Hamming Loss: 0.0373
Validation F1-Score: 0.7533
